In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import random
import datetime as dt
import math
import itertools
from itertools import count
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
import statsmodels.stats.api as sms
from scipy import stats
from scipy.special import logit
import collections
from collections import Counter
from scipy.stats import spearmanr, pearsonr
import pingouin as pg #repeated measure correlation
#plot
from matplotlib.colors import DivergingNorm
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\camil\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.8, the latest is 0.4.0.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\camil\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
import config_origins_chapter0 as config

In [3]:
#choose method to run the script on
name_method = 'ML-method'
#name_method = 'TD-method'

In [4]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
pal_ = config.pal_
pal_zone = config.pal_zone
pal_pens = config.pal_pens
dico_pen_ts = config.dico_pen_ts
pal_tracking_system = config.pal_tracking_system
path_extracted_data_cleaning = os.path.join(path_extracted_data,'Cleaning')
path_extracted_data_cleaning_model = os.path.join(path_extracted_data_cleaning,'model')  
path_extracted_data_cleaning_rawvscleaned = os.path.join(path_extracted_data_cleaning,'Raw&Cleaned')  
path_extracted_data_cleaning_rawvscleaned_MLP = os.path.join(path_extracted_data_cleaning_rawvscleaned, 'MLP')
if not os.path.exists(path_extracted_data_cleaning_rawvscleaned_MLP):
    os.makedirs(path_extracted_data_cleaning_rawvscleaned_MLP)

# Download data

In [ ]:
#var to select hens based on their available tracked days AND to find the estiamted number of transitions
#Note: this dataset ccontain the variable estimates for the processed and unprocessed dataset
df_var = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, name_method+'_daily_ALL_variables.csv'), sep=';', 
                     parse_dates=['level'], dayfirst=True) 
print(df_var.shape)
df_var['tracking_system'] = df_var['PenID'].map(lambda x: dico_pen_ts[x])
df_var['PenID'] = df_var['PenID'].map(lambda x: 'Pen '+str(int(x)))
#keep days with same artificial light (not that due to technical issue, sometimes we loose few hours on specific days for 
#some animals, thus we will remove these)
df_var = df_var[df_var['verification_daily_total_duration']==54000]
#remove the two dates with irregular WG openeing schedule:
li_date2remove = [dt.datetime(2021,1,4), dt.datetime(2020,11,13)] #first one already not here due to health assessment
df_var = df_var[~df_var['level'].isin(li_date2remove)]
#keep hens with at least 54 days (55 drop from 140 to 90)
df_henselection = df_var.groupby('HenID')['level'].agg(lambda x: len(set(x))).reset_index() #set as cleaning & raw
print(df_henselection['level'].value_counts())
#we choose that all use hens have approxiamely same amount of observations, otherwise if a hen have only 9 days, it will 
#sometims acount as much as another hen having 54 days
df_henselection['has_more_54_days'] = df_henselection['level'].map(lambda x: x>=54)
df_henselection['has_more_54_days'].value_counts()
li_hen_2keep = df_henselection[df_henselection['has_more_54_days']]['HenID'].unique()
print((len(li_hen_2keep)))
df_var = df_var[df_var['HenID'].isin(li_hen_2keep)]
print(df_var.shape)
print('You are analysing %d days, starting at: %s until: %s, with %d hens'%(len(df_var['level'].unique()), 
                                                              str(df_var['level'].min()), 
                                                              str(df_var['level'].max()),
                                                              len(df_var['HenID'].unique())))
df_var.head(3)

In [ ]:
#weather conditions
path_weather = os.path.join(r'C:\Users\camil\Desktop\vm_exchange\PhD_Data\Weather','Weather_process.csv')
df_weather_h = pd.read_csv(path_weather, sep=';', parse_dates=['date','Timestamp'])
df_weather_h = df_weather_h.sort_values('Timestamp') 
df_weather_h = df_weather_h.drop(['date'], axis=1)
df_weather_h['Timestamp_weather'] = df_weather_h['Timestamp'].copy()
print(df_weather_h.shape)
display(df_weather_h.head(3))

In [ ]:
#prediction per records
df_pred = pd.read_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned, id_run+name_method+'_Rawdata&Model.csv'), sep=';', 
                 parse_dates=['Timestamp','date'], dayfirst=True)
df_pred.rename(columns={'Trackingsystem_Zone':'Zone'}, inplace=True)
#add when WG is open and select only during the day when wg is open
df_pred['is_WG_open'] = df_pred['Timestamp'].map(lambda x: x.hour in [10,11,12,13,14,15])
df_pred = df_pred[df_pred['HenID'].isin(li_hen_2keep)]
df_pred = df_pred.sort_values('Timestamp') 
df_pred.rename(columns={'date':'level'}, inplace=True)
print(df_pred.shape)
display(df_pred.head(3))
print(min(df_pred['Timestamp']), max(df_pred['Timestamp']))

In [ ]:
#keep only when tags was functioning (i.e. when we computed the var)
print(df_pred.shape, df_var.shape)
df_pred = pd.merge(df_pred, df_var[df_var['type']=='cleaning'][['HenID', 'level','verification_daily_total_duration']], 
                    on=['level','HenID'], how='left')
df_pred = df_pred[~df_pred['verification_daily_total_duration'].isnull()]
print(df_pred.shape)
df_pred.head(3)

In [ ]:
df_pred['prediction'].value_counts(normalize=True)

# Humidity and Temperature - hourly estimated error

In [ ]:
#check variation over the day... exist very much
df_weather_h['date'] = df_weather_h['Timestamp'].map(lambda x: dt.datetime.date(x))
df_wday = df_weather_h[df_weather_h['hour'].isin([11,12,13,14,15,16])].groupby(['date'])[['temperature_C', 'humidity_avgpercent']].agg(lambda x: list(x)).reset_index()
df_wday['humidity_avgpercent_avg'] = df_wday['humidity_avgpercent'].map(lambda x: np.mean(x))
df_wday['temperature_C_avg'] = df_wday['temperature_C'].map(lambda x: np.mean(x))
df_wday

In [ ]:
#merge it to the corresponding (closest) Timestamp and keep only when the WG is open and only the WG registration
#only when WG is open, and only WG registrations (as we have externa weather conditions, not internal, internal is control by
#the barn automatically)
df_pred_weather = pd.merge_asof(df_pred, df_weather_h, on='Timestamp')
df_pred_weather['hour'] = df_pred_weather['Timestamp'].map(lambda x: x.hour)
display(df_pred_weather.tail(3))
df_pred_weather = df_pred_weather[(df_pred_weather['Zone']=='1_Zone') & (df_pred_weather['is_WG_open'])]
print(df_pred_weather.shape)
df_pred_weather[['temperature_C', 'humidity_avgpercent','hour','PenID','Timestamp_weather']].head(3)

In [ ]:
df_quantitatif_weather = df_pred_weather.groupby(['Timestamp_weather','PenID','hour','system'])[['prediction','temperature_C',
                                                            'humidity_avgpercent']].agg(lambda x: list(x)).reset_index()
df_quantitatif_weather['humidity_avgpercent'] = df_quantitatif_weather['humidity_avgpercent'].map(lambda x: x[0])
df_quantitatif_weather['temperature_C'] = df_quantitatif_weather['temperature_C'].map(lambda x: x[0])
df_quantitatif_weather['IFR'] = df_quantitatif_weather['prediction'].map(lambda x: len(x)-sum(x))
df_quantitatif_weather['nbr registrations'] = df_quantitatif_weather['prediction'].map(lambda x: len(x))
df_quantitatif_weather['estimated error rate'] = df_quantitatif_weather.apply(lambda x: x['IFR']/x['nbr registrations'], axis=1)
df_quantitatif_weather['estimated transitions'] = df_quantitatif_weather['prediction'].agg(lambda x: sum(x))
#print(df_quantitatif_weather.shape)
#df_quantitatif_weather = df_quantitatif_weather[df_quantitatif_weather['nbr registrations']>5] #does not chagne results
print(df_quantitatif_weather.shape)
df_quantitatif_weather = df_quantitatif_weather[df_quantitatif_weather['hour']!=17]
print(df_quantitatif_weather.shape)
display(df_quantitatif_weather.head(3))
df_quantitatif_weather.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,
                                               'environmentalfactorestimatederrorrate.csv'), index=False, sep=';')
#sns.histplot(df_quantitatif_weather, x='estimated error rate');
#sns.histplot(df_quantitatif_weather,x='nbr registrations');

## Estimated error within pen-zone areas

In [ ]:
#computing per animal and taking the mean +-std deviation
#IRF/nbr registrations = estimated error rate
#all except zone 1 when wg is close (important to keep nights for nestboxes!)
df_quantitatif = df_pred[~((~df_pred['is_WG_open'])&(df_pred['Zone']=='1_Zone'))].groupby(['PenID','Zone','HenID'])['prediction'].agg(lambda x: (len(x)-sum(x))/len(x)).reset_index()
display(df_quantitatif.head(3))
df_quantitatif = df_quantitatif.groupby(['PenID','Zone'])['prediction'].agg(lambda x: list(x)).reset_index()
df_quantitatif.rename(columns={'prediction':'estimated error rate','Zone':'Trackingsystem_Zone'}, inplace=True)
display(df_quantitatif.head(3))
#mean and std over hens
df_quantitatif['stddev'] = df_quantitatif['estimated error rate'].map(lambda x: np.std(x)) #not IC as not normally distrib.
df_quantitatif['mean'] = df_quantitatif['estimated error rate'].map(lambda x: np.mean(x))
df_quantitatif['annot'] = df_quantitatif.apply(lambda x: str(round(x['mean'],2))+' \n +/-'+str(round(x['stddev'],2)), axis=1)
display(df_quantitatif.head(3))
#annotations
df_annot = df_quantitatif.pivot(values='annot',index='PenID',columns='Trackingsystem_Zone')
print('annotations:')
display(df_annot)
annot_mean_std = df_annot.to_numpy()
#values
df_quantitatif = df_quantitatif.pivot(values='mean',index='PenID',columns='Trackingsystem_Zone')
print('values:')
display(df_quantitatif)
M = df_quantitatif.to_numpy()
dico_x_name = {'1_Zone':'Winter garden',
               '2_Zone':'Litter',
               '3_Zone':'Lower perch',
               '4_Zone':'Nestbox',
               '5_Zone':'Top floor'}
dico_pen_name = {'pen3':'Pen 3',
               'pen4':'Pen 4',
               'pen5':'Pen 5',
               'pen6':'Pen 6',
               'pen8':'Pen 8',
               'pen9':'Pen 9',
               'pen10':'Pen 10',
               'pen11':'Pen 11',
               'pen12':'Pen 12'}
plt.figure(figsize=(8,4))
sns.heatmap(M.transpose(), xticklabels=[dico_pen_name[x] for x in df_quantitatif.index.tolist()], 
            yticklabels=[dico_x_name[x] for x in df_quantitatif.columns.tolist()], cmap=sns.color_palette("Greys"),
            annot=annot_mean_std.transpose(), fmt='', annot_kws={"size":8.4})
plt.title(r'Estimated error rate')
plt.savefig(os.path.join(path_extracted_data_cleaning_rawvscleaned, 'EER_ZonePerPen.png'),format='png',dpi=300,
           bbox_inches='tight')
plt.show()

# amount of transitions

In [ ]:
#only when WG is open to include all zones
#use all except zone 1  when the winter garden is close, as these period can only have FR and no CR

In [ ]:
def nbr_transition(li):
    '''function to compute number of transition from list of consecutives zone name'''
    return max(len(list(itertools.groupby(li, lambda x: x)))-1,0) #max in case of empty list
#small test
#t = [1,1,1,1,2,2,2,3,3,3,4,4,2,2,2,2]
#print(nbr_transition(t)) #4

#only when WG is open to have full barn access
df_pred_ = df_pred[df_pred['is_WG_open']].copy()
#unprocessed
df_tr = df_pred_.groupby(['HenID','level'])['Zone'].agg(lambda x: nbr_transition(list(x))).reset_index()
df_tr.rename(columns={'Zone':'Unprocessed_number_transition'}, inplace=True)
display(df_tr.head(3))

#unproc
df_estiamtedtr = df_pred_[df_pred_['prediction']==1].groupby(['HenID','level'])['Zone'].agg(lambda x: nbr_transition(list(x))).reset_index()
df_estiamtedtr.rename(columns={'Zone':'Estimated_number_transition'}, inplace=True)
display(df_estiamtedtr.head(3))

In [ ]:
#merge processed and unnprocessed amount of transitions per day
df_trall = pd.merge(df_estiamtedtr, df_tr, on=['HenID','level'])
print(df_estiamtedtr.shape, df_tr.shape, df_trall.shape)
df_trall.head(3)

In [ ]:
#all zones are linked: an error in one induce an error in another. we will here look if the overall nbr of estimated correct 
#registrations is linked with the estimated error rate
#to keep all zones and everything equal, we will only look while the WG is open: i.e the full pen is available for the birds
df_transitionlevel = df_pred_.groupby(['HenID','PenID','system','level'])['prediction'].agg(lambda x: list(x)).reset_index()
df_transitionlevel['IFR'] = df_transitionlevel['prediction'].map(lambda x: len(x)-sum(x))
df_transitionlevel['nbr registrations'] = df_transitionlevel['prediction'].map(lambda x: len(x))
df_transitionlevel['EER'] = df_transitionlevel.apply(lambda x: x['IFR']/x['nbr registrations'], axis=1)
#add estimated number of transitions over that day per bird
df_transitionlevel = pd.merge(df_transitionlevel, df_trall, on=['level','HenID'], how='left')
df_transitionlevel.to_csv(os.path.join(path_extracted_data_cleaning_rawvscleaned,
                                               'estimatednumbertransitionsestimatederrorrate.csv'), index=False, sep=';')
print(df_transitionlevel.shape)

In [ ]:
sns.scatterplot(x='Estimated_number_transition', y='EER', data=df_transitionlevel);

In [ ]:
#df_pred[(df_pred['HenID']=='hen_1')&(df_pred['level']==dt.datetime(2020,11,14))].iloc[0:25]